# Amazon Top 50 Bestselling Books 2009 - 2019
该数据从Kaggle上下载

In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *

sc = SparkContext(conf= SparkConf().setMaster('local'))
spark = SparkSession.builder.getOrCreate()

In [79]:
df = pd.read_csv('../data/bestsellers_with_categories.csv')



#这里将价格为0的行去除
index = df[df['Price']==0].index
df2 = df.drop(index, axis=0)
df2

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction


行550 列7的csv表格

In [56]:
df = df.drop
df.describe()

,User Rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


可以看出User Rating的均值在4.6分，最小值为3.3，最大值为4.9


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


## 使用Spark读取数据

In [10]:
#将csv文件上传
!hadoop fs -put /home/spark/code/SparkProgramming-PySpark/data/bestsellers_with_categories.csv /user/spark/data/

In [2]:
sparkdf = spark.read.options(header='True', inferSchema='True', delimiter=',')\
                            .csv('data/bestsellers_with_categories.csv')
sparkdf.show()

+--------------------+--------------------+-----------+-------+-----+----+-----------+
|                Name|              Author|User Rating|Reviews|Price|Year|      Genre|
+--------------------+--------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoo...|            JJ Smith|        4.7|  17350|    8|2016|Non Fiction|
|   11/22/63: A Novel|        Stephen King|        4.6|   2052|   22|2011|    Fiction|
|12 Rules for Life...|  Jordan B. Peterson|        4.7|  18979|   15|2018|Non Fiction|
|1984 (Signet Clas...|       George Orwell|        4.7|  21424|    6|2017|    Fiction|
|5,000 Awesome Fac...|National Geograph...|        4.8|   7665|   12|2019|Non Fiction|
|A Dance with Drag...| George R. R. Martin|        4.4|  12643|   11|2011|    Fiction|
|A Game of Thrones...| George R. R. Martin|        4.7|  19735|   30|2014|    Fiction|
|A Gentleman in Mo...|         Amor Towles|        4.7|  19699|   15|2017|    Fiction|
|A Higher Loyalty:...|         James Comey|

In [3]:
sparkdf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)



### 1.分析得出评分前10的书籍名字及评分

In [6]:
sparkdf.sort(sparkdf['User Rating'].desc()).select('Name', 'User Rating').show(20)

+--------------------+-----------+
|                Name|User Rating|
+--------------------+-----------+
|Goodnight, Goodni...|        4.9|
|Oh, the Places Yo...|        4.9|
|Goodnight, Goodni...|        4.9|
|Brown Bear, Brown...|        4.9|
|Hamilton: The Rev...|        4.9|
|Dog Man: A Tale o...|        4.9|
|Harry Potter and ...|        4.9|
|Dog Man: Brawl of...|        4.9|
|Harry Potter and ...|        4.9|
|Dog Man: For Whom...|        4.9|
|Harry Potter and ...|        4.9|
|Jesus Calling: En...|        4.9|
|Jesus Calling: En...|        4.9|
|Brown Bear, Brown...|        4.9|
|Jesus Calling: En...|        4.9|
|Dog Man: Brawl of...|        4.9|
|Jesus Calling: En...|        4.9|
|Dog Man: Lord of ...|        4.9|
|Last Week Tonight...|        4.9|
|Humans of New Yor...|        4.9|
+--------------------+-----------+
only showing top 20 rows



### 2.分析得出阅读量前十的书籍和阅读量

In [36]:
sparkdf.sort(sparkdf['Reviews'].desc()).select('Name', 'Reviews').show()

+--------------------+-------+
|                Name|Reviews|
+--------------------+-------+
|Where the Crawdad...|  87841|
|The Girl on the T...|  79446|
|The Girl on the T...|  79446|
|            Becoming|  61133|
|            Becoming|  61133|
|           Gone Girl|  57271|
|           Gone Girl|  57271|
|           Gone Girl|  57271|
|The Fault in Our ...|  50482|
|The Fault in Our ...|  50482|
|The Fault in Our ...|  50482|
|The Fault in Our ...|  50482|
|The Nightingale: ...|  49288|
|The Nightingale: ...|  49288|
|Fifty Shades of G...|  47265|
|Fifty Shades of G...|  47265|
|         The Martian|  39459|
|All the Light We ...|  36348|
|All the Light We ...|  36348|
|       The Alchemist|  35799|
+--------------------+-------+
only showing top 20 rows



### 3.分析得出每个作者的总阅读量

In [45]:
rdd = sparkdf.select('Author', 'Reviews').rdd
result = rdd.map(lambda line: (line['Author'], line['Reviews']))\
        .reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], False)
resultDF = result.toDF()
resultDF = resultDF.withColumnRenamed('_1', 'Auhtor')\
    .withColumnRenamed('_2', 'Reviews')

resultDF.show()

+--------------------+-------+
|              Auhtor|Reviews|
+--------------------+-------+
|     Suzanne Collins| 278329|
|          John Green| 210419|
|           E L James| 178011|
|           Dr. Seuss| 176545|
|       Gillian Flynn| 171813|
|       Paula Hawkins| 158892|
|   Laura Hillenbrand| 148365|
|          Harper Lee| 146152|
|        Gary Chapman| 145958|
|     Don Miguel Ruiz| 139848|
|          Eric Carle| 136822|
|       Dale Carnegie| 125005|
|      Michelle Obama| 122266|
|         Sarah Young| 117456|
|       R. J. Palacio| 108125|
|      Kristin Hannah|  98576|
|         Marie Kondō|  90564|
|         Delia Owens|  87841|
|        Roger Priddy|  86615|
|American Psycholo...|  85800|
+--------------------+-------+
only showing top 20 rows



写入数据库

In [47]:
schema = StructType([StructField('Author', StringType(), True),
                    StructField('Reviews', LongType(), True)])

prop = {}
prop['user'] = 'root'
prop['password'] = 'Passwd1!'
prop['driver'] = 'com.mysql.jdbc.Driver'
resultDF.write.jdbc('jdbc:mysql://localhost:3306/spark', 'Amazon_ReviewsTop15', 'append', prop)

### 4.分析各个评分的个数

In [6]:
rdd = sparkdf.select('User Rating').rdd
rdd1 = rdd.map(lambda line: (line['User Rating'], 1)).reduceByKey(lambda a,b: a+b)\
            .sortByKey(True)
rating_countDF = rdd1.toDF()
rating_countDF = rating_countDF.withColumnRenamed('_1', 'User Rating').withColumnRenamed('_2', 'Counts')
rating_countDF.show()

+-----------+------+
|User Rating|Counts|
+-----------+------+
|        3.3|     1|
|        3.6|     1|
|        3.8|     2|
|        3.9|     3|
|        4.0|    14|
|        4.1|     6|
|        4.2|     8|
|        4.3|    25|
|        4.4|    38|
|        4.5|    60|
|        4.6|   105|
|        4.7|   108|
|        4.8|   127|
|        4.9|    52|
+-----------+------+



In [109]:
#保存数据

prop = {}
prop['user'] = 'root'
prop['password'] = 'Passwd1!'
prop['driver'] = 'com.mysql.jdbc.Driver'

rating_countDF.write.jdbc('jdbc:mysql://localhost:3306/spark', 'Rating_counts', 'append', prop)